## Advent of code 2023 day 11-20
See https://adventofcode.com/

In [9]:
# note that this notebook requires the .venv environment (which is set up with pypy3.10-v7.3.13-win64)
# to activate it from a git bash shell: source .venv/Scripts/activate

import collections
import itertools
import functools
import re
import copy
import math
import sys
import time
import json
import heapq
import bisect
import random
import dataclasses

import sortedcontainers

import zio

In [2]:
# version check and timestamp
# NB the timestamp supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}')

python version: 3.10.13 (f1607341da97ff5a1e93430b6e8c4af0ad1aa019, Sep 28 2023, 05:42:24)
[PyPy 7.3.13 with MSC v.1929 64 bit (AMD64)]
# start_ts=1702663725


In [ ]:
# 2023 day 14 part 1
# start_ts=1702594360
# mv ~/Downloads/input* data_src/2023-day-14-input.txt
# big input file looks like: single big map
# idea: part 1 parse as lines, then just simulate the rolling and load calc.

sample2='''
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
'''

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

def move_cell(board, xsrc, ysrc, xdest, ydest):
    board[ydest]=board[ydest][:xdest]+board[ysrc][xsrc]+board[ydest][xdest+1:]
    board[ysrc]=board[ysrc][:xsrc]+'.'+board[ysrc][xsrc+1:]

def roll_north(board):
    '''roll all Os north as far as possible'''
    for x in range(len(board[0])):
        for ydest in range(len(board)):
            if board[ydest][x]=='.': # empty destination spot
                for ysrc in range(ydest+1, len(board)):
                    if board[ysrc][x]=='O': # move it
                        move_cell(board, x, ysrc, x, ydest)
                        break
                    elif board[ysrc][x]=='#': # skip it
                        ydest=ysrc
                        break

def total_load_north(board):
    '''calc noth load (which starts counting south)'''
    res=0
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='O':
                res+=len(board)-y
    return res

board=lines
roll_north(board)
#print('rolled:')
#for line in board:
#    print(line)
score=total_load_north(board)
print(f'part 1: {score=}')

# part 1: 113078

In [ ]:
# 2023 day 14 part 2 implementation A
# idea: for simplicity keep approach from part 1, just introduce a clockwise rotation, do n cycles to start, eg 100,
# after that determine a set of static rocks and each cycle compare the board with the previous one,
# subtract from set of statics,
# for dynamics per location create list of cycles where there's a rock there
# ouch - overlapping cycles so have to track individual rocks through both rolls and rotations :-(

def count_rrocks(board):
    '''return number of (potentially) rolling rocks'''
    n=0
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c=='O':
                n+=1
    return n

def rotate_gr(board):
    '''rotate board 90 degrees clockwise'''
    res=[]
    for x in range(len(board[0])):
        row=''
        for y in range(len(board)-1, -1, -1):
            row+=board[y][x]
        res.append(row)
    return res

def do_cycle(board):
    '''single cycle of rolling in 4 directions'''
    for _ in range(4):
        roll_north(board)
        board=rotate_gr(board)
    return board

def update_movers(oldboard, board, movers, statics, ci):
    '''update tracking of moving rocks, oldboard is prev. board, ci is current cycle index
    (1-based)'''
    for y,row in enumerate(board):
        for x,c in enumerate(row):
            if c!=oldboard[y][x]:
                statics.discard( (x,y) )
            if ((x,y) not in statics) and c=='O':
                cl=movers.setdefault( (x,y) , [])
                cl.append(ci)

def create_future_board(board, movers, statics, ci):
    '''based on tracking show the board of specified ci'''
    # clear all movers, paint in all statics
    for xy in statics:
        x,y=xy
        assert board[y][x]=='O'
    for xy in movers.keys():
        x,y=xy
        board[y]=board[y][:x]+'.'+board[y][x+1:]
    # paint in the right movers
    for xy,mci in movers.items():
        x,y=xy
        period=mci[-1]-mci[-2]
        assert period== mci[-2]-mci[-3]
        if (ci-mci[-1]) % period == 0:
            board[y]=board[y][:x]+'O'+board[y][x+1:]
    return board

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
board=lines
print(f'rrocks={count_rrocks(board)}')

startcyc=100
assert startcyc>(len(board)+len(board[0]))*2
for _ in range(startcyc):
    board=do_cycle(board)
statics=set() # rolling rocks that stay in place
for y,row in enumerate(board):
    for x,c in enumerate(row):
        if c=='O':
            statics.add( (x,y) )
extracyc=startcyc
movers={} # maps location to list of cycle ids when there's a rock there
for ci in range(extracyc):
    oldboard=copy.deepcopy(board)
    board=do_cycle(board)
    update_movers(oldboard, board, movers, statics, startcyc+ci+1)
print(f'after sim rrocks={count_rrocks(board)}')
print(f'{statics=}')
print(f'{movers=}')
board=create_future_board(board, movers, statics, 1000000000)
print(f'future rrocks={count_rrocks(board)}')
print('future board:')
for line in board:
    print(line)
score=total_load_north(board)
print(f'part 2: {score=}')

In [10]:
# 2023 day 14 part 2 implementation B
# track individual rocks through both rolls and rotations :-(

sample2='''
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
'''

def parse_board_b(lines):
    '''convert board as text to:
    unmovs is set of (x,y) of cube-rocks, movs is a map of id (rid) to (x,y) of each round-rock '''
    unmovs=set()
    movs={}
    rid=0
    for y,row in enumerate(lines):
        for x,c in enumerate(row):
            if c=='#':
                unmovs.add( (x,y) )
            elif c=='O':
                movs[rid]= (x,y)
                rid+=1
    return unmovs,movs
    
def roll_north_b(unmovs, movs):
    '''roll all Os/movs north as far as possible'''
    occup=set() # occupied cells
    occup.update(unmovs)
    occup.update(movs.values())
    while True:
        some_moved=False
        for rid in movs.keys():
            xy=movs[rid]
            x,y=xy
            while y>0 and (x,y-1) not in occup:
                occup.remove( (x,y) )
                y-=1
                xy= (x,y)
                occup.add( xy )
                movs[rid]= xy
                some_moved=True
        if not some_moved:
            break

def rotate_b(unmovs, movs):
    '''rotate board (unmovs and movs) 90 degrees clockwise'''
    lastrow=max([ xy[1] for xy in (unmovs | set(movs.values())) ])
    newunmovs=set()
    for xy in unmovs:
        x,y=xy
        newx=lastrow-y
        newy=x
        newunmovs.add( (newx, newy) )
    newmovs={}
    for rid, xy in movs.items():
        x,y=xy
        newx=lastrow-y
        newy=x
        newmovs[rid]= (newx, newy)
    unmovs.clear()
    unmovs.update(newunmovs)
    movs.clear()
    movs.update(newmovs)

def update_tracks_b(movs, tracks, ci):
    '''update tracks for current ci; maps rid to a map of (x,y) to a list of ci'''
    for rid, xy in movs.items():
        tm=tracks.setdefault(rid, {})
        cl=tm.setdefault(xy, [])
        cl.append(ci)

def do_cycle_b(unmovs, movs, tracks, ci):
    '''single cycle of rolling in 4 directions'''
    for _ in range(4):
        roll_north_b(unmovs, movs)
        rotate_b(unmovs, movs)
    update_tracks_b(movs, tracks, ci)

def print_board_b(unmovs, movs, title):
    print(title+':')
    lastcol=max([ xy[0] for xy in (unmovs | set(movs.values())) ])
    lastrow=max([ xy[1] for xy in (unmovs | set(movs.values())) ])
    for y in range(lastrow+1):
        row=''
        for x in range(lastcol+1):
            if (x,y) in unmovs:
                row+='#'
            elif (x,y) in movs.values():
                row+='O'
            else:
                row+='.'
        print(row)
    print()

@dataclasses.dataclass
class XYItem:
    '''helper class to calculate the repeating period for a rock'''
    diffs=[] # list of repeating periods per position
    di: int=None # index into diffs
    total: int=0 # sum(diffs[di:])
    nexttotal: int=0 # sum(diffs[di-1:])

def cycle_future_board_b(unmovs, movs, tracks, ci):
    '''based on tracking create the board of specified ci'''
    movs.clear()
    # analyze tracks, based on that put the movs in the right spot on ci one by one
    for rid, tm in tracks.items():
        xydata=[] # list of XYItem
        for xy, cl in tm.items():
            if len(cl)==1: # 'startup' position that was only visited once and can be ignored
                continue
            assert len(cl)>=6
            diffs=[]
            for i in range(1, len(cl)):
                diffs.append(cl[i]-cl[i-1])
            item=XYItem()
            item.diffs=diffs
            item.di=len(diffs)-1
            item.total=diffs[item.di]
            xydata.append(item)
            print(f'{rid=}, {xy=}: starting on ci={cl[0]}, {diffs=}')
        # find least common cycle length for all positions through which this rock cycles
        while True:
            xydata.sort(key=lambda xyd: xyd.total)
            if xydata[0].total==xydata[-1].total: # all have the same value for total?
                break
            for xyd in xydata:
                xyd.nexttotal=xyd.total+xyd.diffs[xyd.di-1]
            xydata.sort(key=lambda xyd: xyd.nexttotal)
            xyd=xydata[0] # the first element has the lowest nexttotal, so advance that one
            xyd.total=xyd.nexttotal
            xyd.di-=1
        cyclen=xydata[0].total
        print(f'{cyclen=}')
        # TODO now just take ci mod cycle length, list matching xys, should be only one, put in movs
    # TODO
    pass

def total_load_north_b(movs):
    # TODO
    return score

sample1=open('data_src/2023-day-14-input.txt').read()
lines=[s for s in sample2.splitlines() if len(s)>0 ]
unmovs,movs=parse_board_b(lines)
# unmovs is set of (x,y) of cube-rocks, movs is a map of id (rid) to (x,y) of each round-rock
print(f'rrocks={len(movs)}')
tracks={} # maps rid to a map of (x,y) to a list of ci

startcyc=600
assert startcyc>(len(lines)+len(lines[0]))*2
for ci in range(startcyc):
    do_cycle_b(unmovs, movs, tracks, ci+1)
print(f'rrocks={len(movs)}')
print(f'{tracks=}')
cycle_future_board_b(unmovs, movs, tracks, 1000000000)
score=total_load_north_b(movs)
print(f'part 2: {score=}')

rrocks=18
rrocks=18
tracks={0: {(3, 2): [1], (2, 3): [2, 15, 21, 45, 79, 92, 98, 122, 156, 169, 175, 199, 233, 246, 252, 276, 310, 323, 329, 353, 387, 400, 406, 430, 464, 477, 483, 507, 541, 554, 560, 584], (4, 8): [3, 16, 22, 46, 80, 93, 99, 123, 157, 170, 176, 200, 234, 247, 253, 277, 311, 324, 330, 354, 388, 401, 407, 431, 465, 478, 484, 508, 542, 555, 561, 585], (4, 9): [4, 17, 23, 47, 81, 94, 100, 124, 158, 171, 177, 201, 235, 248, 254, 278, 312, 325, 331, 355, 389, 402, 408, 432, 466, 479, 485, 509, 543, 556, 562, 586], (3, 9): [5, 13, 18, 43, 77, 82, 90, 95, 120, 154, 159, 167, 172, 197, 231, 236, 244, 249, 274, 308, 313, 321, 326, 351, 385, 390, 398, 403, 428, 462, 467, 475, 480, 505, 539, 544, 552, 557, 582], (4, 6): [6, 19, 25, 30, 49, 83, 96, 102, 107, 126, 160, 173, 179, 184, 203, 237, 250, 256, 261, 280, 314, 327, 333, 338, 357, 391, 404, 410, 415, 434, 468, 481, 487, 492, 511, 545, 558, 564, 569, 588], (6, 4): [7, 9, 33, 50, 52, 84, 86, 110, 127, 129, 161, 163, 187, 204, 

NameError: name 'score' is not defined

In [ ]:
# 2023 day 13
# start_ts=1702590185
# mv ~/Downloads/input* data_src/2023-day-13-input.txt
# big input file looks like: bunch of maps
# idea: part 1 parse as groups of lines, then just iterate over possible reflection lines
# (instead of having horizontal and vertical reflection checks we use a rotate)

sample2='''
#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#
'''

sample1=open('data_src/2023-day-13-input.txt').read()
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
print(f'ngroups={len(groups)}')

def find_reflect(board):
    '''find reflections across horizontal line, return lines above'''
    res=set()
    for n in range(1, len(board)): # try with n lines above fold
        failed=False
        i=0
        while n-1-i >= 0 and n+i < len(board):
            if board[n-1-i] != board[n+i]:
                failed=True
                break
            i+=1
        if not failed:
            res.add(n)
    return res

def rotate_gr(board):
    '''rotate board 90 degrees clockwise'''
    res=[]
    for x in range(len(board[0])):
        row=''
        for y in range(len(board)-1, -1, -1):
            row+=board[y][x]
        res.append(row)
    return res

# part 1
score=0
for groupi, group in enumerate(groups):
    lines=find_reflect(group)
    #print(f'{groupi=}: {lines=}')
    for n in lines:
        score+=100*n
    rgroup=rotate_gr(group)
    lines=find_reflect(rgroup)
    #print(f'r{groupi=}: {lines=}')
    for n in lines:
        score+=n
print(f'part 1: {score=}')

# part 1: 30802

# part 2
# just try all smudges and discard original lines, easy using sets
score=0
for groupi, group in enumerate(groups):
    lines=find_reflect(group)
    lines2=set()
    for y,row in enumerate(group):
        for x,c in enumerate(row): # try every smudge
            group[y]= group[y][:x]+('.' if c=='#' else '#')+group[y][x+1:]
            lines2.update(find_reflect(group))
            group[y]= group[y][:x]+c+group[y][x+1:]
    #print(f'{groupi=}: {lines=}, {lines2=}')
    lines=lines2-lines
    for n in lines:
        score+=100*n
    group=rotate_gr(group)
    lines=find_reflect(group)
    lines2=set()
    for y,row in enumerate(group):
        for x,c in enumerate(row): # try every smudge
            group[y]= group[y][:x]+('.' if c=='#' else '#')+group[y][x+1:]
            lines2.update(find_reflect(group))
            group[y]= group[y][:x]+c+group[y][x+1:]
    #print(f'r{groupi=}: {lines=}, {lines2=}')
    lines=lines2-lines
    for n in lines:
        score+=n
print(f'part 2: {score=}')

# part 2: 37876

In [ ]:
# 2023 day 12 part 0

def check_nums(s, nums, check_all):
    #print(f'check_nums: {s=}, {nums=}, {check_all=}')
    n=0
    in_dmg=False
    nums_idx=0
    for c in s:
        if in_dmg and c=='#':
            n+=1
        elif (not in_dmg) and c=='.':
            n=0
        elif c=='#':
            n=1
            in_dmg=True
        else:
            assert c=='.'
            if n>0:
                if nums_idx>=len(nums):
                    return False
                if nums[nums_idx]==n:
                    nums_idx+=1
                else:
                    return False
            n=0
    if check_all:
        if n>0:
            if nums_idx>=len(nums):
                return False
            if nums[nums_idx]==n:
                nums_idx+=1
            else:
                return False
        return nums_idx>=len(nums)
    else:
        return True

In [ ]:
# 2023 day 12 part 0
# tests for check_nums

assert check_nums('.###....#', [3, 2, 1], True)==False
assert check_nums('.###....#', [3, 2, 1], False)==True
assert check_nums('.###....##.#', [3, 2, 1], True)==True
assert check_nums('.###....##.#.#.#.#', [3, 2, 1], False)==False

In [ ]:
# 2023 day 12 part 1
# mv ~/Downloads/input* data_src/2023-day-12-input.txt
# big input file looks like: 1000 short lines
# idea: part 1 parse each line as a string and list of numbers, then depth first search on each char in the string

sample2='''
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
'''

def check_arran(s, line, nums):
    if len(s)>=len(line):
        if not check_nums(s, nums, True):
            return 0
        return 1
    else:
        if not check_nums(s, nums, False):
            return 0
        c=line[len(s)]
        if c=='?':
            total=0
            total+=check_arran(s+'.', line, nums)
            total+=check_arran(s+'#', line, nums)
            return total
        else:
            assert c=='.' or c=='#'
            total=check_arran(s+c, line, nums)
            return total

def count_arran(line, nums):
    '''count possible arrangements'''
    cnt=check_arran('', line, nums)
    #print(f'count_arran: {line=}, {cnt=}')
    return cnt

sample1=open('data_src/2023-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[]
for line in lines:
    tup=line.split()
    assert len(tup)==2
    nums=[ int(n) for n in tup[1].split(',') ]
    data.append( (tup[0], nums) )

In [ ]:
# 2023 day 12 part 1 run

total=0
for tup in data:
    total+=count_arran(*tup)
print(f'part 1: {total=}')

# part 1: 7090

In [ ]:
# 2023 day 12 part 2
# idea: DFS as implemented above takes too long, 
# top 3 lines of actual input:
#  .#?#???????.????# 1,2,3,2,1  # cnt=49762
#  ?????????? 1,1,4
#  ????.??.??.??? 1,2
# unfolded already take way too long this way, so a new approach:
# - remove . from start and end, replace .. by ., split on .
# - now DFS on nums, maintaining a remaining list of strings and a multiplication factor
# - if only ? remain in a string, calculate a multiplication factor for ways to put n next nums in those ?
# - cutoff if remaining strings is less than nums+min.separators
# this still takes quite long, so there has to be a faster way but for now this is ok-ish

sample3='''
?###???????? 3,2,1
'''

sample4='''
.#?#???????.????# 1,2,3,2,1
?????????? 1,1,4
????.??.??.??? 1,2
'''

sample5='''
??????#??????.? 2,4,1,1,1
'''

def unfold_tup(line, nums):
    s=line
    nums2=list(nums)
    for _ in range(4):
        s+='?'+line
        nums2.extend(nums)
    while s.startswith('.'):
        s=s[1:]
    while s.endswith('.'):
        s=s[:-1]
    while '..' in s:
        s=s.replace('..', '.')
    return s.split('.'), nums2

#unfold_tup('.#', [1])
#unfold_tup('???.###', [1,1,3])

def count_arran2(substr, si, atsubstart, nums, ni, mult, tslen, tnlen):
    """
    count possible arrangements /fits of nums[ni:] into substr[si:]

    Args:
        substr (list of strings): strings to fit the numbers in (each only contains ? and #)
        si (int): index in substr to current string
        atsubstart (bool): true if at the start of substr[si], false otherwise
        nums (list of int): list of numbers to fit into substr (each a length of damaged springs)
        ni (int): index in nums to current number to match
        mult (int): multiplier of number of arrangements to return
        tslen (int): total length of strings in substr, plus separators, still available
        tnlen (int): minimum total length of nums that still need to be fit

    Returns:
        int: number of different ways to fit
    """
    #print(f'count_arran2: {substr=}, {si=}, {atsubstart=}, {nums=}, {ni=}, {mult=}')
    #assert tslen==sum([len(s) for s in substrs[si:]])+len(substrs[si:])-1
    #assert tnlen==sum(nums[ni:])+len(nums[ni:])-1
    while si<len(substr) and substr[si]=='': # skip any empty substr
        tslen-=1
        si+=1
        atsubstart=True
    if ni>=len(nums): # done, are we good? (only ? remaining)
        for s in substr[si:]:
            if '#' in s:
                return 0        
        return mult
    if si>=len(substr): # all substr processed but nums remaining, not good
        return 0
    if tnlen>tslen: # remaining nums cannot fit
        return 0
    s=substr[si]
    if '#' in s: # match a separator of 1-n (0-n at start), then the next num
        cnt=0
        if atsubstart and len(s)>=nums[ni]:
            substr[si]=s[nums[ni]:]
            cnt+=count_arran2(substr, si, False, nums, ni+1, mult,
                              tslen-nums[ni], tnlen-nums[ni]-1)
        for i in range(0, len(s)-nums[ni]): # e.g. s is 5 chars, nums[ni] is 2 so first 3 can be sep
            if s[i]=='?':
                substr[si]=s[i+1+nums[ni]:]
                cnt+=count_arran2(substr, si, False, nums, ni+1, mult,
                                  tslen-nums[ni]-i-1, tnlen-nums[ni]-1)
            else:
                break
        substr[si]=s
        return cnt
    else: # only ?, update mult, but mind atsubstart, for 0-n nums how many ways?
        if not atsubstart: # have to start w/ a separator
            s=s[1:]
            tslen-=1
            atsubstart=True
        cnt=0
        cnt+=count_arran2(substr, si+1, True, nums, ni, mult,  # skip this substr by putting only sep in it
                          tslen-len(s)-1, tnlen)
        for nnums in range(1, len(nums)-ni+1): # calculate ways for next nnums nums to fit in
            ntot=sum(nums[ni:ni+nnums]) # characters covered by the nums
            nrem=len(s)-ntot-(nnums-1) # remaining after nums and mandatory separators
            if nrem<0: # doesn't fit, can stop here
                break
            # how many ways can we divide nrem marbles over nnums+1 bin?
            # formulated as a stars-and-bars problem:
            # (see https://brilliant.org/wiki/identical-objects-into-distinct-bins/)
            # we have to place nnums bars in nrem+nnums positions, so nrem+nnums over nnums
            ways=math.comb(nrem+nnums, nnums)
            assert ways>0
            cnt+=count_arran2(substr, si+1, True, nums, ni+nnums, mult*ways,
                              tslen-len(s)-1, tnlen-ntot-nnums)
        substr[si]=s
        return cnt

sample1=open('data_src/2023-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

total=0
for line in lines:
    tup=line.split()
    assert len(tup)==2
    nums=[ int(n) for n in tup[1].split(',') ]
    substrs,nums=unfold_tup(tup[0], nums)
    cnt=count_arran2(substrs, 0, True, nums, 0, 1,
                     sum([len(s) for s in substrs])+len(substrs)-1, sum(nums)+len(nums)-1)
    print(f'count_arran2: {substrs=}, {nums=}, {cnt=}')
    total+=cnt
print(f'part 2: {total=}')

# part 2: 6792010726878 (run time: 105 minutes!!)

In [ ]:
# 2023 day 11 part 1
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: big map
# idea: part 1 parse as list of lines, then expand in the lines,
# then convert to a list of coordinates and calculate manhattan distance

sample2='''
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]

# first expand columns
x=0
while x<len(lines[0]):
    foundgal=False
    for y in range(len(lines)):
        if lines[y][x]=='#':
            foundgal=True
    if not foundgal:
        for y in range(len(lines)):
            lines[y]=lines[y][:x]+' '+lines[y][x:]
        x+=2
    else:
        x+=1
# expand rows
y=0
while y<len(lines):
    foundgal='#' in lines[y]
    if not foundgal:
        lines.insert(y, '')
        y+=2
    else:
        y+=1

#print('expanded:')
#for line in lines:
#    print(line)

def manh_dist(a, b):
    '''manhattan distance between two positions, each a tuple of x,y'''
    dist=abs(a[0]-b[0])+abs(a[1]-b[1])
    return dist

def total_dist(data):
    '''based on list of x,y positions calculate total manhattan distance between
    each pair'''
    total=0
    for pair in itertools.combinations(data, 2):
        total+=manh_dist(*pair)
    return total

data=[] # (x,y) of galaxy
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            data.append( (x,y) )
total=total_dist(data)
print(f'part 1: {total=}')

# part 1: 10033566

In [ ]:
# 2023 day 11 part 2
# idea: based on lists of empty row and column numbers, shift/expand each galaxy
# while converting from the map to list of coordinates, then calculate total distance 
# as before

def empty_rows(lines):
    '''return list of empty row numbers'''
    return [ y for y,row in enumerate(lines) if '#' not in row ]

def empty_cols(lines):
    '''return list of empty column numbers'''
    return [ x for x in range(len(lines[0])) if '#' not in { lines[y][x] for y in range(len(lines)) } ]

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
erows=empty_rows(lines)
ecols=empty_cols(lines)
print(f'erows: {erows}')
print(f'ecols: {ecols}')

data=[] # (x,y) of galaxy, expanded
fact=1000000
for y,row in enumerate(lines):
    for x,c in enumerate(row):
        if c=='#':
            newx=x
            for n in ecols:
                if n<x:
                    newx+=fact-1
            newy=y
            for n in erows:
                if n<y:
                    newy+=fact-1
            data.append( (newx,newy) )
total=total_dist(data)
print(f'part 2: {total=}')

In [ ]:
# TEMPLATE
# 2023 day 11
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2023-day-11-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2023-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
groups=zio.get_line_groups(sample1.splitlines(), nostrip=False)
data=[ int(s) for s in lines[0].split(',') ]
data=[ s.split() for s in lines ]
data=[ [cmd, int(num), 0] for cmd, num in data ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed